In [956]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from pandas.core.reshape.merge import merge
import random

## Load Dataset

In [957]:
df = pd.read_csv("data/netflix_titles.csv")
critics = pd.read_csv("data/rotten_tomatoes_critic_reviews.csv")
audience = pd.read_csv("data/rotten_tomatoes_movies.csv")

## Understanding Data

In [958]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [959]:
audience.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


## Cleaning

In [960]:
df.drop(df[df['type'] == 'TV Show'].index, inplace=True)

### Reviewing Columns

In [961]:
df.columns
audience.columns

Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')

In [962]:
audience = audience[['movie_title', 'directors', 'original_release_date',
       'tomatometer_rating', 'audience_rating', 'genres']]
df = df[['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description']]

### Duplicates

In [963]:
df.duplicated().sum()
audience.duplicated().sum()

0

### Null Values

In [964]:
df.isna().sum() / df.shape[0]

type            0.000000
title           0.000000
director        0.030314
cast            0.079226
country         0.042775
date_added      0.000000
release_year    0.000000
rating          0.000930
duration        0.000000
listed_in       0.000000
description     0.000000
dtype: float64

In [965]:
df = df.dropna()

In [966]:
audience = audience.dropna()

## Merge

I can merge the two data sets by connecting individual movies through their title, release date, and director. I can't just merge them through their titles––the name of a movie is not unique to that movie. It could be a remake or a different movie altogether.

Match the names of 'audience' columns to 'df' columns:

In [967]:
audience = audience.rename(columns={"movie_title": "title", "directors": "director"})

For the two lists to merge, the data has to match perfectly. In my 'audience' data, the date is displated "[# year]-[# month]-[# day]". In my 'audience' data, the date is displayed "[name of month] [# day], [# year]".

To make sure the data matches, I am going to create a function that only takes the first four digits of the 'audience' data date. The 'df' data has a separate column which has the release year, so I'll be merging it with that.

In [968]:
def year_of_date(full_date):
    year = full_date[:4]
    return year

audience['release_year'] = audience['original_release_date'].map(year_of_date)

In [969]:
# print(audience['release_year'])
audience.head()

,title,director,original_release_date,tomatometer_rating,audience_rating,genres,release_year
0,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,2010-02-12,49.0,53.0,"Action & Adventure, Comedy, Drama, Science Fic...",2010
1,Please Give,Nicole Holofcener,2010-04-30,87.0,64.0,Comedy,2010
2,10,Blake Edwards,1979-10-05,67.0,53.0,"Comedy, Romance",1979
3,12 Angry Men (Twelve Angry Men),Sidney Lumet,1957-04-13,100.0,97.0,"Classics, Drama",1957
4,"20,000 Leagues Under The Sea",Richard Fleischer,1954-01-01,89.0,74.0,"Action & Adventure, Drama, Kids & Family",1954


Cut out the "original_release_date and "movie_titles" columns from 'audience':

In [970]:
audience = audience[['title', 'director', 'tomatometer_rating', 'audience_rating', 'release_year', 'genres']]

Change the data type of 'audience["release_year"]' to ints instead of strings so we can merge:

In [971]:
audience['release_year'] = audience['release_year'].astype(int)

In [972]:
df.index = df.index - 1
df["release_year"][0]

2016

In [973]:
df_merged = audience.merge(df, how="left", right_on=['title', 'release_year', 'director'], left_on=['title', 'release_year', 'director'])
# print(df_merged['cast'].notna().any())
df_merged.head()

,title,director,tomatometer_rating,audience_rating,release_year,genres,type,cast,country,date_added,rating,duration,listed_in,description
0,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,49.0,53.0,2010,"Action & Adventure, Comedy, Drama, Science Fic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Please Give,Nicole Holofcener,87.0,64.0,2010,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,Blake Edwards,67.0,53.0,1979,"Comedy, Romance",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12 Angry Men (Twelve Angry Men),Sidney Lumet,100.0,97.0,1957,"Classics, Drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"20,000 Leagues Under The Sea",Richard Fleischer,89.0,74.0,1954,"Action & Adventure, Drama, Kids & Family",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I originally merged 'audience' into 'df', but I later realized 'df' doesn't have enough popular titles. For example, 'df' only has one Chris Nolan movie and two Stanley Kubrick movies––two very popular directors. In contrast, 'audience' has all of them. So I went back and created a new dataframe that merges 'df' into 'audience' but doesn't delete all the datapoints that don't merge. This means I can keep the majority of titles, just some won't have more data.

## Input 

Make 'df["title"]' the first column so I can skip over it easier when rotating through columns (I'll do this later on):

Get rid of 'df' columns 'type':

In [974]:
df_merged = df_merged[['title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description', 'genres', 'tomatometer_rating', 'audience_rating']]

Questions:

Name three of your favorite movies.
    collect data, see what these have in common
    if we don't have a movie in the data, we ask for another (or two or three)
Do you tend to side with critics or audiences more?
Are there any specific genres, directors, actors, time periods, ratings you tend to enjoy?

then display movie title, director, actor, plotline, release date, duration


or...I can ask user whether they're looking for a fun movie to watch or a well done movie idk

Changing the 'genres' column from a string to a list.

In [975]:
# most_common_genre = most_common_string(movies, 'genres')
# print(user_movie1[10])

# ChatGPT:

def string_to_list(input_string):
    values = input_string.split(',')
    values = [value.strip() for value in values]
    return values


df_merged['genres'] = df_merged['genres'].apply(string_to_list)

print(df_merged['genres'].head())


0    [Action & Adventure, Comedy, Drama, Science Fi...
1                                             [Comedy]
2                                    [Comedy, Romance]
3                                    [Classics, Drama]
4           [Action & Adventure, Drama, Kids & Family]
Name: genres, dtype: object


Same with the cast:

In [976]:
def string_to_list(input_data):
    if isinstance(input_data, str):  # Check if the input is a string
        return [name.strip() for name in input_data.split(',')]
    else:
        return input_data  # Return the input unchanged if it's not a string

df_merged['cast'] = df_merged['cast'].apply(string_to_list)

print(df_merged['cast'].head())

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: cast, dtype: object


Use MinMax Scaler to get the percentile of each movie's tomatometer rating and save it into a new column:

In [977]:
# # selected_columns = ['tomatometer_rating', 'audience_rating']
scaler = MinMaxScaler()
scaler.fit(df_merged['tomatometer_rating'].values.reshape(-1, 1))

df_scaled = pd.DataFrame(scaler.transform(df_merged['tomatometer_rating'].values.reshape(-1, 1)), columns=['tomatometer_rating_scaled'])
df_merged['tomatometer_rating_scaled'] = df_scaled['tomatometer_rating_scaled']
df_merged['tomatometer_rating_scaled'].head()

0    0.49
1    0.87
2    0.67
3    1.00
4    0.89
Name: tomatometer_rating_scaled, dtype: float64

Now same thing with audience rating:

In [978]:
scaler = MinMaxScaler()
scaler.fit(df_merged['audience_rating'].values.reshape(-1, 1))

df_scaled = pd.DataFrame(scaler.transform(df_merged['audience_rating'].values.reshape(-1, 1)), columns=['audience_rating_scaled'])
df_merged['audience_rating_scaled'] = df_scaled['audience_rating_scaled']
df_merged['audience_rating_scaled'].head()

0    0.53
1    0.64
2    0.53
3    0.97
4    0.74
Name: audience_rating_scaled, dtype: float64

In [1004]:
df_merged.head()

,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,genres,tomatometer_rating,audience_rating,tomatometer_rating_scaled,audience_rating_scaled
0,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,"[Action & Adventure, Comedy, Drama, Science Fi...",49.0,53.0,0.49,0.53
1,Please Give,Nicole Holofcener,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,[Comedy],87.0,64.0,0.87,0.64
2,10,Blake Edwards,NaN,NaN,NaN,1979,NaN,NaN,NaN,NaN,"[Comedy, Romance]",67.0,53.0,0.67,0.53
3,12 Angry Men (Twelve Angry Men),Sidney Lumet,NaN,NaN,NaN,1957,NaN,NaN,NaN,NaN,"[Classics, Drama]",100.0,97.0,1.00,0.97
4,"20,000 Leagues Under The Sea",Richard Fleischer,NaN,NaN,NaN,1954,NaN,NaN,NaN,NaN,"[Action & Adventure, Drama, Kids & Family]",89.0,74.0,0.89,0.74


In [1097]:
recs = []
movies = []

user_movie1 = df_merged.loc[df_merged[df_merged['title'] == input("Movie 1: ")].index.values].iloc[0]
user_movie2 = df_merged.loc[df_merged[df_merged['title'] == input("Movie 2: ")].index.values].iloc[0]
user_movie3 = df_merged.loc[df_merged[df_merged['title'] == input("Movie 3: ")].index.values].iloc[0]
movies = [user_movie1, user_movie2, user_movie3]

# user_movie2 = df_merged.loc[df_merged[df_merged['title'] == "A Clockwork Orange"].index.values].iloc[0]
# movies = [user_movie1, user_movie2]

### Finding director:

In [1098]:
def most_common_string(movies, column_name):
    counts = {}
    
    for datapoint in movies:
        director = datapoint[column_name]
        if director in counts:
            counts[director] += 1
        else:
            counts[director] = 1
    
    most_common = max(counts, key=counts.get)
    
    return most_common

most_common_director = most_common_string(movies, 'director')

### Finding a genre:

In [1099]:
# def filter_classics(genres_list):
#     return [genre for genre in genres_list if genre != "classics"]

# df_merged['genres'] = df_merged['genres'].apply(filter_classics)

# print(df_merged['genres'])


In [1100]:
def most_common_in_list(my_list):
    counts = {}
    
    for item in my_list:
        if item in counts:
            counts[item] += 1
        else:
            counts[item] = 1
    
    most_common = None
    max_count = 0
    for item, count in counts.items():
        if count > max_count:
            most_common = item
            max_count = count
    
    return most_common


all_genres = []

for movie in movies:
    all_genres.extend(movie['genres'])


most_common_genre = most_common_in_list(all_genres)

### Finding a year

In [1101]:
def calculate_average_num(movies, column_name):
    release_years = [movie[column_name] for movie in movies]
    
    return sum(release_years) / len(release_years) if release_years else 0

average_year = calculate_average_num(movies, 'release_year')

### Finding average critics rating (percentile)

In [1102]:
average_critic = calculate_average_num(movies, 'tomatometer_rating_scaled')
print(average_critic)

0.58


### Finding average audience score (percentile):

In [1103]:
# average_audience = calculate_average_num(movies, 'audience_rating_scaled')
# print(average_audience)

### Finding common cast members:

In [1104]:
# all_cast = []

# for movie in movies:
#     all_genres.extend(movie['cast'])

# most_common_cast = most_common_in_list(all_cast)

In [1105]:
# all_countries = []

# for movie in movies:
#     all_genres.extend(movie['country'])

# most_common_country = most_common_in_list(all_countries)

In [1106]:
def has_genre(genres_list):
    return most_common_genre in genres_list

In [1107]:
filtered_movies = df_merged[
    # (df_merged['director'] == most_common_director) & 

    ((df_merged['release_year'] - 10) <= average_year) & 
    (average_year <= (df_merged['release_year'] + 10)) &

    df_merged['genres'].apply(has_genre) &   

    ((df_merged['tomatometer_rating_scaled'] - 0.05) <= average_critic) & 
    (average_critic <= (df_merged['tomatometer_rating_scaled'] + 0.05))
    ]

print(len(filtered_movies))

453


In [1108]:
# if most_common_cast:
#     filtered_movies = [movie for movie in filtered_movies if most_common_cast in movie['cast']]

# if most_common_cast:
#     filtered_movies = [movie for movie in filtered_movies if most_common_country in movie['country']]

# Results:

In [1109]:
filtered_movies_list = filtered_movies.values.tolist()

random_movies = random.sample(filtered_movies_list, 10)

for movie in random_movies:
    print(movie)

['Waiting to Exhale', 'Forest Whitaker', nan, nan, nan, 1995, nan, nan, nan, nan, ['Comedy', 'Drama'], 56.0, 79.0, 0.56, 0.79]
['Felon', 'Ric Roman Waugh', nan, nan, nan, 2008, nan, nan, nan, nan, ['Action & Adventure', 'Drama', 'Mystery & Suspense'], 63.0, 82.0, 0.63, 0.8200000000000001]
['Miracle on 34th Street', 'Les Mayfield', nan, nan, nan, 1994, nan, nan, nan, nan, ['Drama', 'Kids & Family', 'Science Fiction & Fantasy'], 60.0, 62.0, 0.6, 0.62]
['David & Layla', 'Jay Jonroy', nan, nan, nan, 2007, nan, nan, nan, nan, ['Art House & International', 'Comedy', 'Drama', 'Romance'], 53.0, 43.0, 0.53, 0.43]
['Wild Things', 'John McNaughton', nan, nan, nan, 1998, nan, nan, nan, nan, ['Comedy', 'Drama', 'Mystery & Suspense'], 62.0, 53.0, 0.62, 0.53]
['MirrorMask', 'Dave McKean', nan, nan, nan, 2005, nan, nan, nan, nan, ['Drama', 'Kids & Family', 'Science Fiction & Fantasy'], 54.0, 80.0, 0.54, 0.8]
['Permanent Midnight', 'David Veloz', nan, nan, nan, 1998, nan, nan, nan, nan, ['Documentary',